<a href="https://colab.research.google.com/github/abidlifiras/QA_LLM/blob/master/google_flan_t5_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/abidlifiras/QA_LLM.git

Cloning into 'QA_LLM'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 16 (delta 3), reused 6 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (16/16), 611.03 KiB | 1.89 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [1]:
!pip install transformers==4.17 datasets evaluate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 15.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.


In [3]:
import transformers
print(transformers.__version__)

4.17.0


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForMultipleChoice, TrainingArguments, Trainer
import pandas as pd
from tqdm import tqdm
import json
from datasets import Dataset
import numpy as np
import evaluate

In [5]:
# Load datasets
df_train = pd.read_json('QA_LLM/dataset/train.json')
df_dev = pd.read_json('QA_LLM/dataset/dev.json')
df_test = pd.read_json('QA_LLM/dataset/test.json')

In [4]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def format_prompt(question, answers_dict):
    letters = ['a', 'b', 'c', 'd', 'e']
    choices = [f"{l}) {answers_dict.get(l, '')}" for l in letters]
    return f"Question: {question}\n" + "\n".join(choices) + "\nAnswer:"

def get_prediction(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=5)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).lower()
    return response.strip()

# Load test set
df_test = pd.read_json('QA_LLM/dataset/test.json')
letter_to_index = {"a": 0, "b": 1, "c": 2, "d": 3, "e": 4}

correct = 0
results = []

for i, row in tqdm(df_test.iterrows(), total=len(df_test)):
    prompt = format_prompt(row["question"], row["answers"])
    pred = get_prediction(prompt)

    # Prendre uniquement la première lettre présente dans la prédiction
    pred_letter = next((c for c in pred if c in letter_to_index), None)
    true_letter = row["correct_answers"][0]

    is_correct = (pred_letter == true_letter)
    correct += is_correct
    results.append({
        "question": row["question"],
        "predicted": pred_letter,
        "correct": true_letter,
        "success": is_correct
    })

accuracy = correct / len(df_test)
print(f"Accuracy: {accuracy:.2%}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

100%|██████████| 622/622 [08:30<00:00,  1.22it/s]

Accuracy: 32.32%


In [16]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def format_input(example):
    question = example["question"]
    answers = example["answers"]
    correct = example["correct_answers"][0]  # 'a', 'b', etc.

    letters = ['a', 'b', 'c', 'd', 'e']
    choices = [f"{l}) {answers.get(l, '')}" for l in letters]
    prompt = f"Question: {question}\n" + "\n".join(choices) + "\nAnswer:"
    return {
        "input_text": prompt,
        "target_text": correct
    }

# Convert DataFrame to Hugging Face dataset
train_ds = Dataset.from_pandas(df_train)
val_ds = Dataset.from_pandas(df_dev)

# Apply formatting
train_ds = train_ds.map(format_input)
val_ds = val_ds.map(format_input)


loading file https://huggingface.co/google/flan-t5-base/resolve/main/spiece.model from cache at /root/.cache/huggingface/transformers/3672c352f8016689903eb5982ada8724824c7521fe851d2cb2fb8c97d64c1e77.d6f0605ae3d57070be74b4c12206072ab332922acff822e6b5458691dbda7551
loading file https://huggingface.co/google/flan-t5-base/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/75038a7024433f7b51c5384e3a00fd182eedbc4765f04ffb151662e8a0153965.1d3a9d7a332839b438f0d489ad98892ef82082982cdf2adddc251c880c7935c9
loading file https://huggingface.co/google/flan-t5-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/google/flan-t5-base/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/6782dec2ba87f4095a19927661f7eb8863189d2f8d65f1293094e9832cb23d28.a6ade5be9ee4d179c3ae03f26ae924a8473ffd7fc4b15c73138dcc1527b00e62
loading file https://huggingface.co/google/flan-t5-base/resolve/main/tokenizer_config.jso

Map:   0%|          | 0/2171 [00:00<?, ? examples/s]

Map:   0%|          | 0/312 [00:00<?, ? examples/s]

In [17]:
def tokenize(example):
    inputs = tokenizer(
        example["input_text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    targets = tokenizer(
        example["target_text"],
        padding="max_length",
        truncation=True,
        max_length=5
    )
    inputs["labels"] = targets["input_ids"]
    return inputs


train_ds = train_ds.map(tokenize, remove_columns=train_ds.column_names)
val_ds = val_ds.map(tokenize, remove_columns=val_ds.column_names)



Map:   0%|          | 0/2171 [00:00<?, ? examples/s]

Map:   0%|          | 0/312 [00:00<?, ? examples/s]

In [21]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, AutoModelForSeq2SeqLM,DataCollatorForSeq2Seq


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
import os
os.environ["WANDB_DISABLED"] = "true" #to disable automatic Weights & Biases logging

from datasets import load_metric

accuracy_metric = load_metric("accuracy")  # Load accuracy metric from Huggingface datasets

from sklearn.metrics import accuracy_score

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Décodage des prédictions et des labels (on ignore les tokens spéciaux)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Extraction de la première lettre, ou '?' si vide
    def first_letter_or_question_mark(text):
        text = text.strip().lower()
        return text[0] if len(text) > 0 else "?"

    decoded_preds = [first_letter_or_question_mark(p) for p in decoded_preds]
    decoded_labels = [first_letter_or_question_mark(l) for l in decoded_labels]

    # Calcul de l'accuracy
    accuracy = accuracy_score(decoded_labels, decoded_preds)

    return {"accuracy": accuracy}



model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


training_args = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-finetuned-qa",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs",
    fp16=True  # si tu as un GPU avec support
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

loading configuration file https://huggingface.co/google/flan-t5-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ef025c8db1020e9f67512623bfcb3f747602a8f1353b618f842826b3a6e8c93e.f1a741507370eee2f95b3b7642e3092506bf175633a325fe97e453763e1e7ebf
Model config T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
 

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,nan,0.355769
2,0.000000,nan,0.355769
3,0.000000,nan,0.355769


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
***** Running Evaluation *****
  Num examples = 312
  Batch size = 8
/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:1949: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  ctx_manager = autocast(dtype=self.amp_dtype)
Saving model checkpoint to ./flan-t5-finetuned-qa/checkpoint-500
Configuration saved in ./flan-t5-finetuned-qa/checkpoint-500/config.json
Model weights saved in ./flan-t5-finetuned-qa/checkpoint-500/pytorch_mode

TrainOutput(global_step=816, training_loss=0.0, metrics={'train_runtime': 580.5204, 'train_samples_per_second': 11.219, 'train_steps_per_second': 1.406, 'total_flos': 4459824636493824.0, 'train_loss': 0.0, 'epoch': 3.0})